<a href="https://colab.research.google.com/github/jpmachinelearning/U4_S16_Trabajo_Final_2P_PLAZA_A-PADILLA_J/blob/main/src/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyGithub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.7/432.7 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 36.3 MB/s eta 0:00:00


In [ ]:
!pip install PyGithub
from google.colab import files, userdata
from github import Github, Auth, GithubException
import pandas as pd
import os

# ==========================================
# 1. CONFIGURACIÓN
# ==========================================
try:
    TOKEN = userdata.get('GITHUB_TOKEN')
    # Configuración para Git por comandos
    USER_GITHUB = "jpmachinelearning" # Tu usuario de GitHub
    EMAIL_GITHUB = "tu_email@ejemplo.com" # Cambia esto por tu email de GitHub
except userdata.SecretNotFoundError:
    print("❌ Error: Asegúrate de crear el secreto 'GITHUB_TOKEN' en el panel de la izquierda.")

# Repositorio fuente para extracción
REPO_NAME = 'OCA/l10n-spain'

# Repositorio destino para guardar la data (formato para Git)
DEST_REPO_NAME = 'U4_S16_Trabajo_Final_2P_PLAZA_A-PADILLA_J'
REPO_URL = f"https://{TOKEN}@github.com/{USER_GITHUB}/{DEST_REPO_NAME}.git"

# Configuración de archivos
nombre_archivo = 'datos_github.csv'
ruta_local = os.path.join('/content/sample_data', nombre_archivo)

# Límite de registros
LIMITE_REGISTROS = 200

# ==========================================
# 2. CONFIGURACIÓN DE GIT (PULL)
# ==========================================
# Clonamos el repo si no existe, o hacemos pull si ya está
%cd /content/
if not os.path.exists(DEST_REPO_NAME):
    print(f"🚀 Clonando repositorio {DEST_REPO_NAME}...")
    !git clone {REPO_URL}
else:
    print(f"🔄 Sincronizando repositorio (git pull)...")
    %cd {DEST_REPO_NAME}
    !git pull origin main
    %cd /content/

# Configurar identidad de Git
!git config --global user.email "{EMAIL_GITHUB}"
!git config --global user.name "{USER_GITHUB}"

# ==========================================
# 2. CONEXIÓN API (Extracción)
# ==========================================
try:
    auth = Auth.Token(TOKEN)
    g = Github(auth=auth)

    repo = g.get_repo(REPO_NAME)
    print(f"✅ Conectado exitosamente al proyecto: {repo.full_name}")

    data = []

    # ==========================================
    # 3. EXTRACCIÓN DE ISSUES (RF-01)
    # ==========================================
    print(f"\n--- Extrayendo hasta {LIMITE_REGISTROS} Issues ---")
    issues = repo.get_issues(state='all', sort='created', direction='desc')
    count = 0

    for item in issues:
        if count >= LIMITE_REGISTROS: break
        if item.pull_request is not None: continue

        print(f"📥 Procesando Issue #{item.number}...")
        comments = [c.body for c in item.get_comments() if c.body]

        data.append({
            'tipo': 'Issue',
            'numero': item.number,
            'comentarios': " [SEP] ".join(comments)
        })
        count += 1

    # ==========================================
    # 4. EXTRACCIÓN DE PULL REQUESTS (RF-01)
    # ==========================================
    print(f"\n--- Extrayendo hasta {LIMITE_REGISTROS} Pull Requests ---")
    pulls = repo.get_pulls(state='all', sort='created', direction='desc')
    count = 0

    for pr in pulls:
        if count >= LIMITE_REGISTROS: break

        print(f"📥 Procesando PR #{pr.number}...")
        comments = [c.body for c in pr.get_issue_comments() if c.body]

        data.append({
            'tipo': 'Pull Request',
            'numero': pr.number,
            'comentarios': " [SEP] ".join(comments)
        })
        count += 1

    # ==========================================
    # 5. GUARDAR RESULTADOS LOCALMENTE
    # ==========================================
    if not data:
        print("⚠️ No se encontraron datos.")
    else:
        df = pd.DataFrame(data)

        # Guardar en /content/sample_data/datos_github.csv
        df.to_csv(ruta_local, index=False, encoding='utf-8-sig')

        print("-" * 30)
        print(f"✨ RESULTADOS DE LA RECOLECCIÓN:")
        print(f"📁 Guardado local en: {ruta_local}")
        print(f"📊 Total de filas (Issues + PRs): {len(df)}")
        print("-" * 30)

        # ==========================================
        # 6. ENVIAR A REPOSITORIO USANDO GIT PUSH
        # ==========================================
        print(f"\n📦 Moviendo archivo al repositorio y subiendo cambios...")

        # Asegurar que la carpeta data existe en el repo clonado
        repo_path = f"/content/{DEST_REPO_NAME}/data"
        os.makedirs(repo_path, exist_ok=True)

        # Copiar de sample_data al repositorio
        !cp {ruta_local} {repo_path}/{nombre_archivo}

        # Comandos de Git para subir
        %cd /content/{DEST_REPO_NAME}
        !git add data/{nombre_archivo}
        !git commit -m "Actualización de datos_github.csv via Git Pull/Push desde Colab"
        !git push origin main

        print(f"\n✅ Proceso completado. Archivo disponible en GitHub y localmente en sample_data.")
        %cd /content/

except Exception as e:
    print(f"❌ Error: {e}")

# ==========================================
# 6. ENVIAR A REPOSITORIO (CON MANEJO DE CONFLICTOS)
# ==========================================
print(f"\n🚀 Sincronizando y subiendo cambios a GitHub...")

# Ruta del repo y carpeta data
repo_path = f"/content/{DEST_REPO_NAME}"
data_folder = os.path.join(repo_path, "data")
os.makedirs(data_folder, exist_ok=True)

# Copiar el archivo fresco de sample_data al repo clonado
!cp {ruta_local} {data_folder}/{nombre_archivo}

%cd {repo_path}

# 1. Preparar el commit local
!git add data/{nombre_archivo}
# Usamos un commit con fecha para diferenciar
import datetime
timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
!git commit -m "Auto-update data: {timestamp}"

# 2. Intentar traer cambios remotos con Rebase
# La estrategia '-X ours' le dice a Git: "Si hay conflicto en una línea,
# mantén lo que acabo de generar en este Colab".
print("🔄 Revisando si hay cambios externos (Pull Rebase)...")
status = os.system("git pull --rebase -X ours origin main")

if status == 0:
    # 3. Empujar los cambios
    print("📤 Subiendo cambios finales...")
    push_status = os.system("git push origin main")
    if push_status == 0:
        print(f"✅ ¡Éxito! Datos sincronizados en el repositorio.")
    else:
        print("❌ Error al hacer push. Revisa permisos del Token.")
else:
    print("⚠️ Hubo un problema al sincronizar. Intentando limpiar estado de Git...")
    !git rebase --abort

%cd /content/

✅ Conectado exitosamente al proyecto: OCA/l10n-spain

--- Extrayendo hasta 200 Issues ---
📥 Procesando Issue #4793...
📥 Procesando Issue #4760...
📥 Procesando Issue #4743...
📥 Procesando Issue #4681...
📥 Procesando Issue #4677...
📥 Procesando Issue #4666...
📥 Procesando Issue #4652...
📥 Procesando Issue #4650...
📥 Procesando Issue #4649...
📥 Procesando Issue #4642...
📥 Procesando Issue #4641...
📥 Procesando Issue #4639...
📥 Procesando Issue #4637...
📥 Procesando Issue #4632...
📥 Procesando Issue #4631...
📥 Procesando Issue #4626...
📥 Procesando Issue #4623...
📥 Procesando Issue #4620...
📥 Procesando Issue #4618...
📥 Procesando Issue #4615...
📥 Procesando Issue #4613...
📥 Procesando Issue #4605...
📥 Procesando Issue #4604...
📥 Procesando Issue #4603...
📥 Procesando Issue #4596...
📥 Procesando Issue #4591...
📥 Procesando Issue #4589...
📥 Procesando Issue #4586...
📥 Procesando Issue #4550...
📥 Procesando Issue #4534...
📥 Procesando Issue #4507...
📥 Procesando Issue #4497...
📥 Procesando I

In [ ]:
import nltk
import string
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [ ]:
def preprocesado(documento):
  documento = documento.lower()
  tokens_nltk = nltk.word_tokenize(documento)
  tags = nltk.pos_tag(tokens_nltk)
  puntuacion = ['--', '_', ':', ')', "'", '"--', ',', ']', '!', '"', '?', '-', '--------', '[', ';', '.', '(', '----------', "'--", '&', "''", '``']
  tokens_nltk_limpio = [word for word, tag in tags if tag not in puntuacion]
  tokens_nltk_limpio = [t.strip('_') for t in tokens_nltk_limpio]
  if '[' in tokens_nltk_limpio: tokens_nltk_limpio.remove('[')
  if '[' in tokens_nltk_limpio: tokens_nltk_limpio.remove(']')
  stop_words = set(stopwords.words('spanish'))
  #stop_words.remove('no') para remover stop_words
  #stop_words.update('así') para añadir stop_words
  tokens_nltk_limpio = [w for w in tokens_nltk_limpio if w not in stop_words and w.isalnum()]
  del tokens_nltk, tags, puntuacion
  return tokens_nltk_limpio

In [ ]:
file_path = '/content/sample_data/datos_github_proyecto.csv'
#data = pd.read_csv(file_path, sep='\s+', header=None)
data = pd.read_csv(file_path)
data

,tipo,numero,titulo,descripcion,comentarios,total_comentarios
0,Issue,33154,Redundant execution of check_class_weight_bal...,### Describe the bug and give evidence about i...,@atheendre130505 \ncan i work on this ??\n,1
1,Issue,33152,⚠️ CI failed on Unit tests (last failure: Jan...,**CI failed on [Unit tests ](https://github.co...,Jan 29: `RuntimeError` in [Linux x86-64 pylat...,2
2,Issue,33146,Using `BisectingKMeans` with `init` raises a p...,"When using `BisectingKMeans`, `n_clusters` gre...","Thanks for the report, I think this is indeed ...",3
3,Issue,33143,RFC Metric `pos_label` handling for multiclass...,> [!WARNING]\n> This is a request for comment ...,NaN,0
4,Issue,33137,Analyze the practical relevance importance of ...,The following paper ran many hyper-parameter t...,NaN,0
...,...,...,...,...,...,...
95,Pull Request,33095,Added intutive explanation for linear regressi...,Added a plain -English sentence to the Ordinar...,NaN,0
96,Pull Request,33092,DOC: clarify that Lasso.path() defaults to Ela...,<!--\r\n🙌 Thanks for contributing a pull reque...,NaN,0
97,Pull Request,33091,DOC/MNT Little clean up around the splitting d...,Noticed while working on #33089\r\n\r\n#### Wh...,"I made the error message more specific, since ...",5
98,Pull Request,33090,CI remove scipy<1.16.2 requirement in wheel bu...,<!--\r\n🙌 Thanks for contributing a pull reque...,NaN,0


In [ ]:
data.count()

,0
tipo,100
numero,100
titulo,100
descripcion,100
comentarios,60
total_comentarios,100


In [ ]:
data.shape

(100, 6)